In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
from lxml import etree

In [7]:
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [8]:
def get_matches(query, choices, limit=2):
    results = process.extract(query, choices, limit=limit)
    return results

# Webscraping

In [3]:
gazette_url = "https://www.thegazette.co.uk/all-notices/London/notice?results-page-size=100&categorycode-all=all&numberOfLocationSearches=1&start-publish-date=1859-01-01&location-distance-1=1&sort-by=oldest-date&text=%22Office+of+the+Inspector-General%22+%22gold%22&end-publish-date=1860-01-01"

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

In [5]:
response = requests.get(gazette_url, headers=headers)

In [6]:
response

<Response [200]>

In [7]:
page = response.content

In [1]:
#page

In [10]:
soup = BeautifulSoup(page, "html.parser")

In [11]:
dom = etree.HTML(str(soup))

In [12]:
testing = dom.xpath('//*[@id="item-"]/div/header/h3/a')

In [13]:
len(testing)

51

# PDF Scraping

In [1]:
import tabula as tb
import pandas as pd
import re

In [2]:
file = "1858_gold_silver_imex.pdf"

In [11]:
!java -version

java version "1.8.0_331"
Java(TM) SE Runtime Environment (build 1.8.0_331-b09)
Java HotSpot(TM) 64-Bit Server VM (build 25.331-b09, mixed mode)


## imports 1858

In [4]:
imports1 = tb.read_pdf(file, pages = "1", area = (165, 0, 375, 800), pandas_options={'header': None}, stream=True)[0]
#imports1

In [5]:
for k in range(1,51):
    globals()['import%s' % k] = tb.read_pdf(file, pages = k, area = (165, 0, 375, 800), 
                                            pandas_options={'header': None}, stream=True)[0].set_index(0)

In [9]:
#create empty dictionary to be filled with {page number:import values row}
import_dict = {}

for k in range(1,51):
    #get index list for the import dataframe on page k
    index_list = globals()['import%s' % k].index.tolist()

    #get top two fuzzy matches from the index list
    row_options = get_matches("Importations", index_list)

    #organize fuzzy matches to access easier
    tuples = []
    for var in row_options:
        tuples.append(var[0])

    #create dict with matches and their fuzz ratios and get the key of the higher value
    #may have to improve this for page 10 issue; something better to compare than fuzz ratio??
    row_fuzzratio_dict = {fuzz.ratio("Importations",tuples[0]): tuples[0], 
                          fuzz.ratio("Importations",tuples[1]): tuples[1]}
    max_key = max(row_fuzzratio_dict, key=row_fuzzratio_dict.get)

    #get row values from highest fuzz ratio match row
    total_value_row = globals()['import%s' % k].loc[row_fuzzratio_dict[max_key]].tolist()
    
    #add the import values row to the import_dict dictionary
    import_dict[k] = total_value_row

In [10]:
## need to fix this so i dont have to delete this page then re-add it!!!!
del import_dict[10]

In [376]:
## shouldnt need this anymore because only worried about the last element in each list
## originally did this to get the gold values too but this is unnecessary thanks to Einzig
# for key,value in import_dict.items():
#     for x,y in enumerate(value):
#         k = str(y)
#         import_dict[key][x] = k
#     z = [x for x in value if x != 'nan']
#     import_dict.update({key: z})

In [11]:
#create empty string to hold the silver import value of each week
int_list_of_silver_import_values = []

for key in import_dict:
    
    last_element = (len(import_dict[key])-1)
    
    #remove the commas from the last value of the import values row and convert it to int
    silver_import_value = import_dict[key][last_element]
    comma_less_value = silver_import_value.replace("," , "")
    silver_import_int_value = int(comma_less_value)
    
    #add the int value of silver import value from each week to the originally empty list
    int_list_of_silver_import_values.append(silver_import_int_value)
 
#sum all the values of the list to get the total silver import value for the whole year
#sum(int_list_of_silver_import_values)

In [17]:
sum(int_list_of_silver_import_values) +201105

6241224

## exports 1858

In [384]:
exports1 = tb.read_pdf(file, pages = "1", area = (490, 0, 720, 800), pandas_options={'header': None}, stream=True)[0]
#exports1

In [385]:
exports2 = tb.read_pdf(file, pages="2", area = (490, 0, 720, 800), pandas_options={'header': None}, stream=True)[0]
#exports2